In [4]:
def add(a: int, b: int) -> int:
    return a + b

def multiply(a: int, b:int) -> int:
    return a*b

tools = [add, multiply]

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import langchain_core, langchain_ollama

#langchain_core : 0.3.9
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)

langchain_core : 0.3.9
langchain_ollama : 0.2.0


In [16]:
# ResponseError: gemma2:9b does not support tools
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

In [17]:
llm_with_tools = llm.bind_tools(tools)

In [18]:
llm_with_tools.invoke("What's 3 * 12?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-11T08:41:05.475892Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': '3', 'b': '12'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 2008940667, 'load_duration': 27557334, 'prompt_eval_count': 210, 'prompt_eval_duration': 1032915000, 'eval_count': 22, 'eval_duration': 946235000}, id='run-0c18e8f1-2ee5-4b40-8878-db740dfab793-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '12'}, 'id': '7ccecee4-67e7-4bf6-a2d3-9076e3a668dd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 210, 'output_tokens': 22, 'total_tokens': 232})

In [22]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': '3', 'b': '12'},
  'id': 'bf033424-08d1-4218-9cd2-1525379c50b8',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': '11', 'b': '49'},
  'id': '1cf02c6b-08af-4c9b-a804-a9fb75521122',
  'type': 'tool_call'}]

### Pydantic class

In [30]:
from pydantic import BaseModel, Field

class add_pydantic(BaseModel):
    a: int
    b: int

class multiply_pydantic(BaseModel):
    a: int
    b: int

tools = [add_pydantic, multiply_pydantic]

In [31]:
# ResponseError: gemma2:9b does not support tools
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

In [32]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools.invoke("What's 3 * 12?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-11T09:06:31.744451Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply_pydantic', 'arguments': {'a': 3, 'b': 12}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1652467209, 'load_duration': 27565000, 'prompt_eval_count': 216, 'prompt_eval_duration': 546011000, 'eval_count': 25, 'eval_duration': 1077748000}, id='run-ecaab7ae-377c-4574-be0c-a801e6ab9806-0', tool_calls=[{'name': 'multiply_pydantic', 'args': {'a': 3, 'b': 12}, 'id': '758d04c9-28fb-49a2-9942-6d5c8e4b267b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 25, 'total_tokens': 241})

In [33]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply_pydantic',
  'args': {'a': '3', 'b': '12'},
  'id': '9e15f61b-2a80-4548-8d37-0edf4165b356',
  'type': 'tool_call'},
 {'name': 'add_pydantic',
  'args': {'a': '11', 'b': '49'},
  'id': '33b0d82a-74b5-4e98-9bbf-88f54f5b75c1',
  'type': 'tool_call'}]

### Parsing

In [39]:
from langchain_core.output_parsers import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools = [add_pydantic, multiply_pydantic])

In [40]:
query = "What is 3 * 12? Also, what is 11 + 49?"
chain.invoke(query)

[multiply_pydantic(a=3, b=12), add_pydantic(a=11, b=49)]